In [43]:
import os
import pandas as pd
import sqlite3

class HolidaySQLconn:

    def __init__(self):
        self.hol_path = "./hols/"
        self.db_path = "./holiday.db"
        self.col_name_list = ["year","month","date","holiday", "holiday_type", "country"]
        self.holiday_df = self.hols_to_df(self.col_name_list)
              

    def hols_to_df(self, col_name_list):
        # 將hol檔案轉成self.holiday_df: pd.dataframe
        holiday_df  = pd.DataFrame(columns=col_name_list)

        for file_name in os.listdir(self.hol_path):
            if ".hol" not in file_name:
                continue
            with open(self.hol_path+file_name, 'r', encoding='utf-16') as file:
                data = file.read()

            holiday_datas = data.split("\n")

            # holiday_datas[0] = [AUHQ_2023節日假期表] 21
            country_name = holiday_datas[0].split("_")[0].split("[")[1]

            for data in holiday_datas[1:]:
                # data = '開國紀念日(放假),2023/1/1'
                holiday_name = data.split(",")[0].split("(")[0]
                holiday_type = data.split(",")[0].split("(")[1].split(")")[0]

                time = data.split(",")[1]

                year = time.split("/")[0]
                month = time.split("/")[1]
                date = time.split("/")[2]

                holiday_df.loc[len(holiday_df )] = [year, month, date, holiday_name, holiday_type, country_name]
        
        return holiday_df
       

    def create_table_by_df_col(self, table_name, col_name):
        # 做一個table，pk為col_name
        create_country_table_cmd = \
            f'''CREATE TABLE {table_name}
                ({col_name} text PRIMARY KEY)'''
        
        #  執行create_country_table_cmd
        try:
            conn = sqlite3.connect(self.db_path)         
            c = conn.cursor()
            c.execute(create_country_table_cmd)
            conn.commit()
        
        except Exception as e:
            print(e)

        finally:
            conn.close()                   
        
    
    def insert_data_by_df_col(self, table_name, col_name):
        # 將self.holiday_df內的country_name資料，塞入country table
        unique_array = pd.unique(self.holiday_df[col_name]) # 取得所有不重複的country_name

        # 用迴圈執行update_country_table_cmd將country_array資料寫入country table
        for unique_value in unique_array:
            try:
                update_cmd = f"INSERT INTO {table_name} ({col_name}) VALUES ('{unique_value}')"
                conn = sqlite3.connect(self.db_path)        
                c = conn.cursor()
                c.execute(update_cmd)
                conn.commit()
            except Exception as e:
                print(e)
            finally:
                conn.close()
                
    def select_table_by_df_col(self, table_name):
        # 執行select指令，存入變數country_table，確認資料有塞進去
        select_cmd = f"SELECT * FROM {table_name}"
        
        try:
            conn = sqlite3.connect(self.db_path)        
            c = conn.cursor()
            table_data = c.execute(select_cmd).fetchall()
            
        except Exception as e:
            print(e)
            
        finally:
            conn.close()

        return table_data

    def create_hol_table(self):
        # 用self.holiday_df內的各col建立正規化的各table
        self.table_name_list = [ col_name+"_table" for col_name in self.col_name_list]
        for table_name, col_name in zip(self.table_name_list, self.col_name_list):
            self.create_table_by_df_col( table_name, col_name)
            self.insert_data_by_df_col( table_name, col_name)
            self.select_table_by_df_col( table_name)

        # 依self.holiday_df內的資料，以及前面正規化的各table，用fk建立table
        try:
            conn = sqlite3.connect(self.db_path)        
            c = conn.cursor()
            c.execute('''CREATE TABLE holiday
                    (Year text,
                    Month text,
                    Date text,
                    holiday_name text,
                    holiday_type text,
                    Country_name text,
                    PRIMARY KEY (Year, Month, Date, holiday_name, Country_name),
                    FOREIGN KEY (Year) REFERENCES year_table(year),
                    FOREIGN KEY (Month) REFERENCES month_table(month),
                    FOREIGN KEY (Date) REFERENCES date_table(date),
                    FOREIGN KEY (holiday_name) REFERENCES holiday_table(holiday),
                    FOREIGN KEY (Country_name) REFERENCES country_table(country))''')
            conn.commit()

        except Exception as e:
            print(e)
        
        finally:
            conn.close()

    def insert_hol_data(self):
        # 將self.holiday_df內的資料，塞入holiday table
        for index, row in self.holiday_df.iterrows():
            try:
                update_cmd = f"INSERT INTO holiday VALUES ('{row['year']}', '{row['month']}', '{row['date']}', '{row['holiday']}', '{row['holiday_type']}', '{row['country']}')"
                conn = sqlite3.connect(self.db_path)         
                c = conn.cursor()
                c.execute(update_cmd)
                conn.commit()
            except Exception as e:
                print(e)
            finally:
                conn.close()
    
    def select_hol_table(self, country=None, year=None, month=None):

        select_cmd = "SELECT * FROM holiday WHERE "

        # 依條件執行select指令，存入變數holiday_table，確認資料有塞進去
        if country is not None:
            country_conditions = [f"Country_name = '{c}'" for c in country]
            select_cmd += "(" + " OR ".join(country_conditions) + ")" + "  AND "

        if year is not None:
            year_conditions = [f"Year = '{y}'" for y in year]
            select_cmd += "(" + " OR ".join(year_conditions) + ")" + "  AND "

        if month is not None:
            month_conditions = [f"Month = '{m}'" for m in month]
            select_cmd += "(" + " OR ".join(month_conditions) + ")" + "  AND "


        # 若where或and結尾，則去除
        if "WHERE" or "AND" in select_cmd[-6:]:
            select_cmd = select_cmd[:-6]
        
        print(select_cmd)

        holiday_table = None

        try:
            conn = sqlite3.connect(self.db_path)         
            c = conn.cursor()
            holiday_table = c.execute(select_cmd).fetchall()
            print(holiday_table)
        
        except Exception as e:
            print(e)

        finally:
            conn.close()

        return holiday_table




In [44]:
if __name__ == '__main__':
    conn_obj = HolidaySQLconn()
    conn_obj.create_hol_table()
    conn_obj.insert_hol_data()
    conn_obj.select_hol_table(year=[2023,2024], month=[2,3,4], country=["AUHQ","AUCN"])
    # conn_obj.select_table_by_df_col("holiday_type"+"_table")

table year_table already exists
UNIQUE constraint failed: year_table.year
[('2023',)]
table month_table already exists
UNIQUE constraint failed: month_table.month
UNIQUE constraint failed: month_table.month
UNIQUE constraint failed: month_table.month
UNIQUE constraint failed: month_table.month
UNIQUE constraint failed: month_table.month
UNIQUE constraint failed: month_table.month
UNIQUE constraint failed: month_table.month
[('1',), ('4',), ('5',), ('6',), ('9',), ('10',), ('2',)]
table date_table already exists
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint fai

In [60]:
conn_obj.conn.close()

In [59]:
conn_obj = HolidaySQLconn()

table year_table already exists
UNIQUE constraint failed: year_table.year
[('2023',)]
table month_table already exists
UNIQUE constraint failed: month_table.month
UNIQUE constraint failed: month_table.month
UNIQUE constraint failed: month_table.month
UNIQUE constraint failed: month_table.month
UNIQUE constraint failed: month_table.month
UNIQUE constraint failed: month_table.month
UNIQUE constraint failed: month_table.month
[('1',), ('4',), ('5',), ('6',), ('9',), ('10',), ('2',)]
table date_table already exists
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint failed: date_table.date
UNIQUE constraint fai

OperationalError: table holiday already exists

In [22]:
conn_obj.hols_to_df()
conn_obj.create_country_table()
conn_obj.insert_country_data()
conn_obj.check_country_table()
conn_obj.create_hol_type_table()
conn_obj.insert_hol_type_data()
conn_obj.check_hol_type_table()
conn_obj.create_hol_name_table()
conn_obj.insert_hol_name_data()
conn_obj.check_hol_name_table()

table country already exists
UNIQUE constraint failed: country.country_name
UNIQUE constraint failed: country.country_name
[('AUCN',), ('AUHQ',)]
table holiday_type already exists
[(1, '放假'), (2, '补班'), (3, '補假'), (4, '公司請假1天'), (5, '放假'), (6, '补班'), (7, '補假'), (8, '公司請假1天'), (9, '放假'), (10, '补班'), (11, '補假'), (12, '公司請假1天')]
table holiday_name already exists
[(1, '元旦'), (2, '农历除夕'), (3, '春节年初一'), (4, '春节年初二'), (5, '春节年初三'), (6, '春节年初四'), (7, '春节年初五'), (8, '春节年初六'), (9, '春节'), (10, '清明节'), (11, '劳动节'), (12, '端午节'), (13, '中秋节'), (14, '国庆节'), (15, '開國紀念日'), (16, '調整休假'), (17, '農曆除夕'), (18, '春節年初一'), (19, '春節年初二'), (20, '春節年初三'), (21, '除夕及春節'), (22, '和平紀念日'), (23, '兒童節'), (24, '民族掃墓節'), (25, '勞動節'), (26, '端午節'), (27, '中秋節'), (28, '國慶日'), (29, '元旦'), (30, '农历除夕'), (31, '春节年初一'), (32, '春节年初二'), (33, '春节年初三'), (34, '春节年初四'), (35, '春节年初五'), (36, '春节年初六'), (37, '春节'), (38, '清明节'), (39, '劳动节'), (40, '端午节'), (41, '中秋节'), (42, '国庆节'), (43, '開國紀念日'), (44, '調整休假'), (45, '農曆除夕'), (46, '春節年初一'), (47,

In [5]:
conn_obj.holiday_df


,Year,Month,Date,holiday_name,holiday_type,Country_name
0,2023,1,1,元旦,放假,AUCN
1,2023,1,2,元旦,放假,AUCN
2,2023,1,21,农历除夕,放假,AUCN
3,2023,1,22,春节年初一,放假,AUCN
4,2023,1,23,春节年初二,放假,AUCN
5,2023,1,24,春节年初三,放假,AUCN
6,2023,1,25,春节年初四,放假,AUCN
7,2023,1,26,春节年初五,放假,AUCN
8,2023,1,27,春节年初六,放假,AUCN
9,2023,1,28,春节,补班,AUCN


In [8]:
country_table

[(1, 'AUCN'), (2, 'AUHQ')]